<a href="https://colab.research.google.com/github/AmoghTantradi/CS182-final-project/blob/main/Token_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import nltk
from nltk.corpus import gutenberg
from nltk.tokenize import word_tokenize
from collections import Counter
import gensim.downloader as api
from sklearn.metrics.pairwise import cosine_similarity
model = api.load("word2vec-google-news-300")

from transformers import CLIPFeatureExtractor, CLIPTextModel, CLIPTokenizer
pretrained_model_name_or_path = "stabilityai/stable-diffusion-2"
tokenizer = CLIPTokenizer.from_pretrained(
    pretrained_model_name_or_path,
    subfolder="tokenizer",
)

def get_rarities(tokenizer):
    # Download the Gutenberg Corpus (if not already downloaded)
    nltk.download('gutenberg')
    # Download the Punkt tokenizer models
    nltk.download('punkt')
    # Load the Gutenberg Corpus
    corpus = gutenberg.raw('bible-kjv.txt')  # You can choose a specific Gutenberg text

    # Tokenize the entire corpus
    tokens = word_tokenize(corpus)

    # Calculate token frequencies
    token_frequencies = Counter(tokens)

    # Calculate relative rarity for each token
    total_tokens = len(tokens)
    relative_rarity = {token: freq / total_tokens for token, freq in token_frequencies.items()}

    # Sort tokens by rarity (from rarest to most common)
    sorted_tokens_by_rarity = sorted(relative_rarity.items(), key=lambda x: x[1], reverse=True)

    return sorted_tokens_by_rarity


[==================================================] 100.0% 1662.8/1662.8MB downloaded


tokenizer/tokenizer_config.json:   0%|          | 0.00/824 [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.06M [00:00<?, ?B/s]

tokenizer/merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer/special_tokens_map.json:   0%|          | 0.00/460 [00:00<?, ?B/s]

In [ ]:
def get_similarities(tokens, category):
  similarities = []
  if category not in model:
    print("ERROR: THE CATEGORY IS NOT IN THE VOCABULARY")
    return
  for token_pair in tokens:
    token = token_pair[0]


    # Calculate the cosine similarity between the word vectors
    if token not in model:
      similarities.append((token, 1)) #If the word is not present, we will not choose it, so we set maximum cosine similarity
    else:

      cosine_sim = cosine_similarity([model[token]], [model[category]])

      similarities.append((token, 1+cosine_sim.item()))
  #normalize similarities:
  total_sim = 0
  for i in range(len(similarities)):
    total_sim += similarities[i][1]
  for i in range(len(similarities)):
    similarities[i] = (similarities[i][0], similarities[i][1]/total_sim)

  #sort by similarity:

  return similarities


In [ ]:
def composite_token_ranking(token_rarities, token_similarities):
  composite_ranking = []
  n = len(token_rarities)
  for i in range(n):
    token = token_rarities[i][0]
    rarity = token_rarities[i][1]
    similarity = token_similarities[i][1]
    composite_ranking.append((token, rarity + similarity + ((rarity*similarity)**2)))
  return composite_ranking


In [ ]:
def find_tokens(category, num_tokens):
  token_rarities = get_rarities(tokenizer)
  token_similarities = get_similarities(token_rarities, category)
  composite_ranking = composite_token_ranking(token_rarities, token_similarities)
  composite_ranking = sorted(composite_ranking, key=lambda x: x[1])
  return composite_ranking[:num_tokens]

In [ ]:
def find_tokens_two_categories(category1, category2, num_tokens):
  tokens1 = find_tokens(category1, num_tokens)
  tokens2 = find_tokens(category2, num_tokens)
  two_token_ranking = []
  n = len(tokens1)
  for i in range(n):
    for j in range(n):
      token1 = tokens1[i][0]
      token2 = tokens2[j][0]
      cosine_sim = cosine_similarity([model[token1]], [model[token2]]).item()
      two_token_ranking.append((token1, token2, cosine_sim))
  two_token_ranking = sorted(two_token_ranking, key=lambda x: x[2])
  return two_token_ranking[:num_tokens]

In [ ]:
'''
How to use: replace "bear" with the class of the object that Stable Diffusion is bring trained to generated.
'''
class_name = "bear"
find_tokens(class_name, 10)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


[('Jesurun', 4.528891410656284e-05),
 ('cor', 4.749721856729523e-05),
 ('grinders', 4.756776541308057e-05),
 ('Jonan', 4.803936641167003e-05),
 ('Raca', 4.817057437068276e-05),
 ('Shalim', 4.8493901142631196e-05),
 ('rie', 4.893973115245141e-05),
 ('Edar', 4.9036059572561385e-05),
 ('Fifth', 4.91512629559453e-05),
 ('Benoni', 4.9185071651618596e-05)]

In [ ]:
'''
How to use: replace arguments with the two categories. If they are the same, category, put the same word twice.
'''

find_tokens_two_categories("cat", "dog", 10)

[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package gutenberg to /root/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


[('backbone', 'KINGS', -0.06793107092380524),
 ('Distributing', 'childbearing', -0.047295942902565),
 ('Distributing', 'Jesurun', -0.04324890673160553),
 ('foresaw', 'KINGS', -0.0410400815308094),
 ('employment', 'KINGS', -0.03265289589762688),
 ('backbone', 'Jesurun', -0.03237507492303848),
 ('Distributing', 'heady', -0.018940895795822144),
 ('Reach', 'childbearing', -0.017750369384884834),
 ('employment', 'heady', -0.013148588128387928),
 ('foreseeing', 'KINGS', -0.011834153905510902)]